# Большие таблицы в Метрике
В этом блоке мы научимся решать очень распространенную проблему выгрузки больших таблиц.

Это связано с тем, что в любой системе есть ограничения по количеству строк в отчете. API Метрики за один запрос может вернуть максимум 100 тысяч строк. Для больших сайтов есть множество отчетов, которые многократно превосходят этот лимит.

Например, для одного нашего проекта выгрузка отчета по поисковым запросам, страницам входа и дате визита за полгода заняла на диске более 6 гигабайт.

Однако, и для небольших сайтов существуют отчеты, когда приходится выгружать частями. Например, когда вам необходимо детально проанализировать посещения сайта по минутам или секундам и разбивке по дополнительной метрике (например, источникам трафика). Такие задачи часто встречаются при анализе оффлайновых рекламных кампаний.

В текущем блоке мы научимся выгружать отчет любого размера, используя параметры limit (необходимое количество строк в ответе, максимум 100 тысяч) и offset (номер первой строки выборки). Этот метод полностью аналогичен тому как мы выгружали список пользователей ВКонтакте.

Будем выгружать отчет по точному времени визита:

<img src='./module12_files/m8_b5_report.jpg'>

Как обычно задаем параметры выгрузки. В качестве измерения (строк отчета) будет выступать время визита ym:s:dateTime. Из метрик для простоты оставим только визиты:

In [1]:
import requests
from yaml import load
from pprint import pprint

In [6]:
with open('./module12_files/yandex_config.yaml', 'r') as f:
    config = load(f)
    token = config['access_token']

API_URL = 'https://api-metrika.yandex.ru/stat/v1/data'
startDate = '2018-02-12'
endDate = '2018-02-18'
counter = '21075004'
metrics = ['ym:s:visits']
dimensions = ['ym:s:dateTime']

# Стандартные преобразования:

metrics_string = ','.join(metrics)
dimensions_string = ','.join(dimensions)

params = {
    'date1': startDate,
    'date2': endDate,
    'id': counter,
    'dimensions': dimensions_string,
    'metrics': metrics_string,
    'oauth_token': token
}

r = requests.get(API_URL, params = params)
data = r.json()

In [8]:
pprint(data)

{'data': [{'dimensions': [{'name': '2018-02-13 23:25:33'}], 'metrics': [2.0]},
          {'dimensions': [{'name': '2018-02-14 18:52:15'}], 'metrics': [2.0]},
          {'dimensions': [{'name': '2018-02-15 09:35:29'}], 'metrics': [2.0]},
          {'dimensions': [{'name': '2018-02-15 11:57:33'}], 'metrics': [2.0]},
          {'dimensions': [{'name': '2018-02-16 09:45:21'}], 'metrics': [2.0]},
          {'dimensions': [{'name': '2018-02-16 18:14:22'}], 'metrics': [2.0]},
          {'dimensions': [{'name': '2018-02-17 18:16:37'}], 'metrics': [2.0]},
          {'dimensions': [{'name': '2018-02-17 22:22:21'}], 'metrics': [2.0]},
          {'dimensions': [{'name': '2018-02-18 09:19:28'}], 'metrics': [2.0]},
          {'dimensions': [{'name': '2018-02-12 00:03:14'}], 'metrics': [1.0]},
          {'dimensions': [{'name': '2018-02-12 00:12:21'}], 'metrics': [1.0]},
          {'dimensions': [{'name': '2018-02-12 00:24:00'}], 'metrics': [1.0]},
          {'dimensions': [{'name': '2018-02-12 00:35

По умолчанию Метрика возвращает первые 100 строк. Добавим параметры limit и offset к запросу.

Для наглядности выведем первые 5 строк и обернем процесс выгрузки в отдельную функцию:

In [9]:
def report_for_offset_value(offset, n_rows):

    """Выгрузка n_rows строк отчета Метрики для заданного значения offset"""   

    params = {
        'date1': startDate,
        'date2': endDate,
        'id': counter,
        'dimensions': dimensions_string,
        'metrics': metrics_string,
        'oauth_token': token,
        'offset': offset,
        'limit': n_rows
    }
    
    r = requests.get(API_URL, params = params)
   
    return r.json()

In [13]:
n_rows, offset = 5, 1 #offset должен быть >= 1

data = report_for_offset_value(offset, n_rows)
pprint(data)

{'data': [{'dimensions': [{'name': '2018-02-13 23:25:33'}], 'metrics': [2.0]},
          {'dimensions': [{'name': '2018-02-14 18:52:15'}], 'metrics': [2.0]},
          {'dimensions': [{'name': '2018-02-15 09:35:29'}], 'metrics': [2.0]},
          {'dimensions': [{'name': '2018-02-15 11:57:33'}], 'metrics': [2.0]},
          {'dimensions': [{'name': '2018-02-16 09:45:21'}], 'metrics': [2.0]}],
 'data_lag': 71,
 'max': [2.0],
 'min': [2.0],
 'query': {'attribution': 'Last',
           'auto_group_size': '1',
           'currency': 'RUB',
           'date1': '2018-02-12',
           'date2': '2018-02-18',
           'dimensions': ['ym:s:dateTime'],
           'group': 'Week',
           'ids': [21075004],
           'limit': 5,
           'metrics': ['ym:s:visits'],
           'offline_window': '21',
           'offset': 1,
           'quantile': '50',
           'sort': ['-ym:s:visits']},
 'sample_share': 1.0,
 'sample_size': 2458,
 'sample_space': 2458,
 'sampled': False,
 'total_rows':

In [14]:
for line in data['data']:
    time_of_visit = line['dimensions'][0]['name']
    visits = line['metrics'][0]   

    print(time_of_visit, visits)

2018-02-13 23:25:33 2.0
2018-02-14 18:52:15 2.0
2018-02-15 09:35:29 2.0
2018-02-15 11:57:33 2.0
2018-02-16 09:45:21 2.0


# Замечание по лимитам выгрузки
Первые 5 строк мы получили. Как определить сколько раз придется выгружать отчет? Посмотрим какой ответ отдает Метрика при очень большом параметре offset. Судя по веб-интерфейсу в нашем отчете 2 449 строк, поэтому посмотрим ответ при offset = 2 500:

In [15]:
pprint(report_for_offset_value(2500, 5))

{'data': [],
 'data_lag': 104,
 'max': [2.0],
 'min': [1.0],
 'query': {'attribution': 'Last',
           'auto_group_size': '1',
           'currency': 'RUB',
           'date1': '2018-02-12',
           'date2': '2018-02-18',
           'dimensions': ['ym:s:dateTime'],
           'group': 'Week',
           'ids': [21075004],
           'limit': 5,
           'metrics': ['ym:s:visits'],
           'offline_window': '21',
           'offset': 2500,
           'quantile': '50',
           'sort': ['-ym:s:visits']},
 'sample_share': 1.0,
 'sample_size': 2458,
 'sample_space': 2458,
 'sampled': False,
 'total_rows': 2449,
 'total_rows_rounded': False,
 'totals': [2458.0]}


In [22]:
pprint(report_for_offset_value(1, 10))

{'data': [{'dimensions': [{'name': '2018-02-13 23:25:33'}], 'metrics': [2.0]},
          {'dimensions': [{'name': '2018-02-14 18:52:15'}], 'metrics': [2.0]},
          {'dimensions': [{'name': '2018-02-15 09:35:29'}], 'metrics': [2.0]},
          {'dimensions': [{'name': '2018-02-15 11:57:33'}], 'metrics': [2.0]},
          {'dimensions': [{'name': '2018-02-16 09:45:21'}], 'metrics': [2.0]},
          {'dimensions': [{'name': '2018-02-16 18:14:22'}], 'metrics': [2.0]},
          {'dimensions': [{'name': '2018-02-17 18:16:37'}], 'metrics': [2.0]},
          {'dimensions': [{'name': '2018-02-17 22:22:21'}], 'metrics': [2.0]},
          {'dimensions': [{'name': '2018-02-18 09:19:28'}], 'metrics': [2.0]},
          {'dimensions': [{'name': '2018-02-12 00:03:14'}], 'metrics': [1.0]}],
 'data_lag': 79,
 'max': [2.0],
 'min': [1.0],
 'query': {'attribution': 'Last',
           'auto_group_size': '1',
           'currency': 'RUB',
           'date1': '2018-02-12',
           'date2': '2018-02-

Лист 'data' оказался пустым, это и есть наше условие выхода из цикла выгрузки.

In [32]:
%%time

condition = True
n_rows = 1000
sum_visits = 0
offset = 1

while True:
#     print(offset)
    data = report_for_offset_value(offset, n_rows)['data']
    if data:
        for line in data:
            sum_visits += line['metrics'][0]
#             print(sum_visits)
        offset += n_rows
    else:
        break

Wall time: 1.09 s


In [31]:
sum_visits

2458.0